## Data Loading and Cleaning

In [1]:
import pandas as pd
import geopandas as gpd
from fuzzywuzzy import process
import geopandas as gpd
import plotly.express as px
import missingno as msno
import matplotlib.pyplot as plt

In [2]:
# Load Airbnb, ZHVI, and NYC geo-spatial datasets
airbnb_data = pd.read_csv('assets/Airbnb_Open_Data.csv', low_memory=False)
zhvi_data = pd.read_csv('assets/ZHVI_dataset.csv')
nyc_geo_data = gpd.read_file('assets/nyc_geo_export.shp')

# Original number of records
airbnb_original_records = len(airbnb_data)
zhvi_original_records = len(zhvi_data)

# Drop rows with missing values in the 'neighbourhood' column
airbnb_data.dropna(subset=['neighbourhood'], inplace=True)
# Calculate difference in records after dropping missing values
airbnb_dropped_records = airbnb_original_records - len(airbnb_data)
print("Airbnb data: Dropped {} records due to missing 'neighbourhood' values.".format(airbnb_dropped_records))

# Remove dollar signs and commas from the 'price' column and convert it to float
airbnb_data['price'] = airbnb_data['price'].replace('[\$,]', '', regex=True).astype(float)

# Convert 'last review' column to datetime format
airbnb_data['last review'] = pd.to_datetime(airbnb_data['last review'], errors='coerce')

# Keep only rows where the 'price' is greater than 0
airbnb_data = airbnb_data[airbnb_data['price'] > 0]
# Calculate difference in records after filtering valid prices
airbnb_price_filtered_records = airbnb_original_records - len(airbnb_data)
print("Airbnb data: Dropped {} records with price less than or equal to zero.".format(airbnb_price_filtered_records))

# Remove duplicate records from Airbnb data
airbnb_data.drop_duplicates(inplace=True)
# Calculate difference in records after removing duplicates
airbnb_duplicates_dropped = airbnb_original_records - len(airbnb_data)
print("Airbnb data: Dropped {} duplicate records.".format(airbnb_duplicates_dropped))

# Drop unnecessary columns from Airbnb data
airbnb_columns_to_keep = ['id', 'neighbourhood', 'price', 'last review', 'number of reviews', 'availability 365']
airbnb_data = airbnb_data[airbnb_columns_to_keep]

#  Drop unnecessary columns from nyc_geo_data
geo_columns_to_keep = ['borough', 'name', 'geometry']
nyc_geo_data = nyc_geo_data[geo_columns_to_keep]

# Create a new DataFrame 'ny_zhvi_data' that contains only rows where the 'City' column is 'New York'
ny_zhvi_data = zhvi_data[zhvi_data['City'] == 'New York'].copy()
zhvi_dropped_records = zhvi_original_records - len(ny_zhvi_data)
print("ZHVI data: Filtered for New York City. Dropped {} records.".format(zhvi_dropped_records))

Airbnb data: Dropped 16 records due to missing 'neighbourhood' values.
Airbnb data: Dropped 262 records with price less than or equal to zero.
Airbnb data: Dropped 803 duplicate records.
ZHVI data: Filtered for New York City. Dropped 21475 records.


### Standardizing Neighborhood Names
- Standardize neighborhood names across datasets by identifying discrepancies, applying fuzzy matching to find best matches, and creating a rename map for correction.
- Update the names in the datasets, print the rename map, and output the number of discrepancies before and after standardization for analysis.

In [3]:
# Function to find discrepancies between two sets of neighbourhood names
def find_discrepancies(source_names, target_names):
    """
    Finds discrepancies between two sets of neighbourhood names.
    
    Args:
        source_names (set): Set of names to compare.
        target_names (set): Set of names to compare against.
        
    Returns:
        list: List of names present in source_names but not in target_names.
    """
    discrepancies = [value for value in source_names if value not in target_names]
    return discrepancies

# Function to normalize neighbourhood names for consistency
def normalize_names(names):
    """
    Normalizes neighbourhood names for consistency.
    
    Args:
        names (iterable): Iterable of names to be normalized.
        
    Returns:
        set: Set of normalized names (lowercased and stripped of leading/trailing whitespace).
    """
    return {name.strip().lower() for name in names}

# Function to find the best matching name for discrepancies using fuzzy matching
def find_best_matches(discrepancies, target_names):
    """
    Finds the best matching name for discrepancies using fuzzy matching.
    
    Args:
        discrepancies (list): List of discrepancy names to be matched.
        target_names (set): Set of names to search for matches.
        
    Returns:
        dict: A dictionary where keys are original discrepancy names and values are their best matching names found in the reference set.
    """
    # Create a dictionary where keys are the normalized (lowercased and stripped of leading/trailing whitespace) names from target_names
    # and values are the original names from target_names
    original_form = {name.strip().lower(): name for name in target_names}

    # Initialize an empty dictionary to store the best matches for discrepancies
    matches = {}

    # Iterate through each discrepancy name
    for name in discrepancies:
        # Use the process.extractOne function to find one best match for the current discrepancy name 
        # within the keys of the original_form dictionary
        # The score_cutoff parameter ensures that only matches with a similarity score above 90 are considered
        best_match = process.extractOne(name, original_form.keys(), score_cutoff=90)
        
        # If a best match is found  add it to the matches dictionary
        if best_match:
            # Store the original name corresponding to the best match in the matches dictionary
            matches[name] = original_form[best_match[0]]

    # Return the matches dictionary containing the best matches for discrepancies
    return matches

# Normalize names in all datasets
airbnb_names = normalize_names(airbnb_data['neighbourhood'].unique())
zhvi_names = normalize_names(ny_zhvi_data['RegionName'].unique())
nyc_names = normalize_names(nyc_geo_data['name'].unique())

# Identify discrepancies after initial normalization
discrepancies_airbnb = find_discrepancies(airbnb_names, nyc_names)
discrepancies_zhvi = find_discrepancies(zhvi_names, nyc_names)

# Find best matches for discrepancies
matches_airbnb = find_best_matches(discrepancies_airbnb, nyc_names)
matches_zhvi = find_best_matches(discrepancies_zhvi, nyc_names)

# Create a rename map for correcting names in the datasets
rename_map = {**matches_airbnb, **matches_zhvi}

# Apply rename map to correct neighbourhood names
airbnb_data['neighbourhood'] = airbnb_data['neighbourhood'].apply(lambda x: rename_map.get(x.strip().lower(), x))
ny_zhvi_data['RegionName'] = ny_zhvi_data['RegionName'].apply(lambda x: rename_map.get(x.strip().lower(), x))

# Re-identify unique neighbourhoods and discrepancies after name corrections
airbnb_names_updated = normalize_names(airbnb_data['neighbourhood'].unique())
zhvi_names_updated = normalize_names(ny_zhvi_data['RegionName'].unique())

discrepancies_airbnb_updated = find_discrepancies(airbnb_names_updated, nyc_names)
discrepancies_zhvi_updated = find_discrepancies(zhvi_names_updated, nyc_names)

# Print the rename map 
print("Rename Map:")
for original_name, corrected_name in rename_map.items():
    print(f"Original: {original_name} -> Corrected: {corrected_name}")

# Output the number of discrepancies after standardizing names
print("\nDataset Discrepancies")
print(f"Initial Airbnb Discrepancies: {len(discrepancies_airbnb)} Updated: {len(discrepancies_airbnb_updated)}")
print(f"Initial ZHVI Discrepancies: {len(discrepancies_zhvi)} Updated: {len(discrepancies_zhvi_updated)}")

Rename Map:
Original: ditmars steinway -> Corrected: steinway
Original: chelsea, staten island -> Corrected: chelsea
Original: highbridge -> Corrected: high  bridge
Original: bay terrace, staten island -> Corrected: bay terrace
Original: bedford-stuyvesant -> Corrected: bedford stuyvesant
Original: mariners harbor -> Corrected: mariner's harbor
Original: tremont -> Corrected: east tremont
Original: harlem -> Corrected: east harlem
Original: east morrisania -> Corrected: morrisania
Original: richmondtown -> Corrected: richmond town
Original: bull's head -> Corrected: bulls head
Original: jamaica -> Corrected: jamaica estates
Original: prospect-lefferts gardens -> Corrected: prospect lefferts gardens
Original: flushing -> Corrected: downtown flushing
Original: corona -> Corrected: south corona
Original: flatiron district -> Corrected: flatiron
Original: downtown brooklyn -> Corrected: downtown
Original: battery park -> Corrected: battery park city
Original: throggs neck -> Corrected: thr

### Temporal Analysis of Airbnb Reviews
- Generated histogram of Reviews by Year, observed significant trend/change in review numbers over years.
- Focus investigation on last 5 years (2019-2023).
- Filter dataset for reviews within this period.
- Filter Zillow's Home Value Index (ZHVI) data for same 5-year period for comparative analysis.

In [4]:
# Extract the year from the 'last review' column and create a new column 'last_review_year'
airbnb_data['last_review_year'] = airbnb_data['last review'].dt.year

fig = px.histogram(airbnb_data, x='last_review_year',
                   title='Histogram of Reviews by Year',
                   labels={'last_review_year': 'Year', 'count': 'Number of Reviews'})


fig.update_layout(
    xaxis=dict(title='Year'),
    yaxis=dict(title='Number of Reviews'),
    showlegend=False
)
fig.show()

# Define the valid year range
valid_year_range = (2019, 2023)

# Filter the DataFrame to keep only rows with valid years
airbnb_data_filtered = airbnb_data[
    (airbnb_data['last_review_year'] >= valid_year_range[0]) &
    (airbnb_data['last_review_year'] <= valid_year_range[1])
]

# Filter Zillow DataFrame for last 5 years 
last_five_years = ny_zhvi_data.columns[-60:]  
zhvi_data_filtered = ny_zhvi_data[['RegionName'] + list(last_five_years)]

### Data Merging
- Merge Airbnb data with NYC Geo data.
- Merge with ZHVI data.
- Drop redundant columns. 
- Drop rows with zero price for any date.
- Saved resulting dataset to CSV file.

In [6]:
# Merge Airbnb data with NYC Geo data
project_data = pd.merge(airbnb_data_filtered, nyc_geo_data, left_on='neighbourhood', right_on='name', how='inner') # how='inner' in both merges to keep only the common neighborhoods.

# Drop redundant column 'name' from NYC Geo data
project_data.drop(['name'], axis=1, inplace=True)

# Merge with ZHVI data
project_data = pd.merge(project_data, zhvi_data_filtered, left_on='neighbourhood', right_on='RegionName', how='inner')

# Drop redundant column 'RegionName' from ZHVI data
project_data.drop(['RegionName'], axis=1, inplace=True)

# Drop rows where the price is 0 for any date
project_data = project_data.dropna(subset=[col for col in project_data.columns if '/' in col])

# Reset index
project_data.reset_index(drop=True, inplace=True)
project_data.head()
# project_data.to_csv('assets/project_data.csv', index = False)

,id,neighbourhood,price,last review,number of reviews,availability 365,last_review_year,borough,geometry,1/31/2019,...,3/31/2023,4/30/2023,5/31/2023,6/30/2023,7/31/2023,8/31/2023,9/30/2023,10/31/2023,11/30/2023,12/31/2023
0,1001254,Kensington,966.0,2021-10-19,9.0,286.0,2021.0,Brooklyn,POINT (-73.98042 40.64238),705411.7989,...,684456.3353,672106.3885,668261.4733,668539.7621,669269.6315,668423.4587,664570.6277,656191.0275,649729.4505,647670.1667
1,1144382,Kensington,786.0,2019-05-16,45.0,NaN,2019.0,Brooklyn,POINT (-73.98042 40.64238),705411.7989,...,684456.3353,672106.3885,668261.4733,668539.7621,669269.6315,668423.4587,664570.6277,656191.0275,649729.4505,647670.1667
2,1221151,Kensington,779.0,2019-05-19,82.0,131.0,2019.0,Brooklyn,POINT (-73.98042 40.64238),705411.7989,...,684456.3353,672106.3885,668261.4733,668539.7621,669269.6315,668423.4587,664570.6277,656191.0275,649729.4505,647670.1667
3,1409486,Kensington,422.0,2019-01-21,20.0,181.0,2019.0,Brooklyn,POINT (-73.98042 40.64238),705411.7989,...,684456.3353,672106.3885,668261.4733,668539.7621,669269.6315,668423.4587,664570.6277,656191.0275,649729.4505,647670.1667
4,1487360,Kensington,887.0,2019-01-02,62.0,115.0,2019.0,Brooklyn,POINT (-73.98042 40.64238),705411.7989,...,684456.3353,672106.3885,668261.4733,668539.7621,669269.6315,668423.4587,664570.6277,656191.0275,649729.4505,647670.1667
